# Replication of Allen Institute's _ecephys_spike_sorting_ repository using SpikeInterface

As explained in "Level of Support" section here https://github.com/AllenInstitute/ecephys_spike_sorting/tree/main, it is generally recommended to use SpikeInterface instead of Allen's original pipeline which is no longer under development and does not support more recent sorters.

This notebook collects the recommended SpikeInterface implementations of modules from Allen's _ecephys_spike_sorting_.

### 1) _extract_from_npx_:
Converts continuous data from raw NPX/NPX2 format (75% compression ratio) to .dat files required for spike sorting and other downstream analysis. Deprecated: the NPX format is no longer used by Open Ephys (or any other software), so this module can be skipped.

### 2) _depth_estimation_:
Creates a JSON file with information about the DC offset of each channel, as well as the channel closest to the brain surface (determined from LFP data). This information is needed to perform the median subtraction step and to run kilosort modules.

#### SpikeInterface implementation:

__detect_bad_channels()__ can be used to detect which channels are outside the brain, as well as channels that have abnormally high levels of noise.

This function returns both the __bad_channel_ids__ and __channel_labels__, which can be __good__, __noise__, __dead__, or __out__ (outside of the brain). These can then be removed from the recording so they are ignored by the spike sorter:

In [ ]:
from spikeinterface.preprocessing import detect_bad_channels

# detect noisy, dead, and out-of-brain channels
bad_channel_ids, channel_labels = detect_bad_channels(recording)
rec_clean = recording.remove_channels(remove_channel_ids=bad_channel_ids)

### 3) _median_subtraction_:
Calls a binary executable that removes the DC offset and common-mode noise from the AP band continuous file.

Because noise on Neuropixels probes is highly correlated across sites that share an ADC, we compute the median of every 24th channel, rather than using the median across all sites. This ends up creating a residual on the order of a few microvolts for large spikes, which can appear in the mean waveform. However, this is well below the probe's noise floor, and shouldn't affect spike sorting or data analysis.

#### SpikeInterface implementation:

Performing median subtraction (and other preprocessing steps) is much easier with SpikeInterface. For Neuropixels data, we recommend using the "phase shift" to align the samples across channels, followed by median subtraction for the whole probe. This removes noise more effectively than just computing the median for simultaneously sampled channels.

Here is the code needed to preprocess Neuropixels data:

In [ ]:
import spikeinterface.full as si

# read Open Ephys dataset
raw_rec = si.read_openephys('/path/to/data',
                            block_index=0,
                            stream_name='ProbeA-AP')

# or SpikeGLX, by changing one line of code:
raw_rec = si.read_spikeglx('/path/to/data', 
                           stream_name='imec0.ap',     
                           load_sync_channel=False)

# perform high-pass filtering (especially important for Neuropixels 2.0 data):
rec_filt = si.highpass_filter(raw_rec, 
                              freq_min=400.)

# remove channels outside the brain
bad_channel_ids, channel_labels = si.detect_bad_channels(rec_filt)
rec_cleaned = rec_filt.remove_channels(bad_channel_ids)

# perform the phase shift (similar to IBL destriping or `tshift` option in CatGT):
rec_shifted = si.phase_shift(rec_cleaned)

# subtract the median across all channels
rec = si.common_reference(rec_shifted, 
                          operator="median", 
                          reference="global")
# The rec object can now be passed to the run_sorter() function to perform spike sorting.

In [ ]:
# read binary file if recording with OpenEphys ONIX system + Bonsai
raw_rec = si.read_binary('/path/to/data', sampling_frequency=sampling_frequency, dtype=dtype, num_channels=num_channels)

### 4) _kilosort_helper_:
Generates config files for Kilosort (versions 2 or 2.5) and launches spike sorting via the Matlab engine. This module auto-generates the channel map, configuration file, and master file for Kilosort, and runs everything via the Matlab engine for Python.

#### SpikeInterface implementation:

SpikeInterface makes it much easier to run the spike sorting step, which only requires a single line of code. We recommend running Kilosort in a Docker container to avoid the need for a Matlab license or complex installation procedures.

After you've installed Docker, you can run Kilosort on a pre-loaded and pre-processed Recording object by running:

In [ ]:
import spikeinterface.full as si

sorting = run_sorter(sorter_name='kilosort2_5', 
                     recording=recording,
                     output_folder="/tmp/kilosort", 
                     docker_image=True)

### 5) _kilosort_postprocessing_:

Clean up Kilosort outputs by removing putative double-counted spikes.

Kilosort occasionally fits a spike template to the residual of another spike. See https://github.com/MouseLand/Kilosort2/issues/29 for more information.

This module aims to correct for this by removing spikes from the same unit or neighboring units that occur within 5 samples (0.16 ms) of one another. This is not ideal, since it can potentially remove legitimate spike times, but on the whole it seems worth it to avoid having spurious zero-time-lag correlation between units.

We are not currently taking into account spike amplitude when removing spikes; the module just deletes one spike from an overlapping pair that occurs later in time.

#### SpikeInterface implementation:

Putative double-counted spikes can be deleted using the __remove_duplicated_spikes__ method in the curation module:

In [ ]:
from spikeinterface.curation import remove_duplicated_spikes

# returns a new sorting object with putative double-counted spikes removed
cleaned_sorting = remove_duplicated_spikes(sorting=sorting, 
                                           censored_period=0.3, # in ms
                                           method='keep_first') # determines which spike to remove

### 6) _noise_templates_:
Identifies noise units based on their waveform shape and ISI histogram.

Kilosort2 generates templates of a fixed length (2 ms) that matches the time coures of an extracellularly detected spike waveform. However, there are no constraints on template shape, which means that the algorithm often fits templates to voltage fluctuations that could not physically result from the current flow associated with an action potential. The units associated with these templates are considered "noise," and must be filtered out prior to analysis. This is true for other spike sorters as well, but the characteristics of the noise waveforms may be highly algorithm-dependent.

#### SpikeInterface implementation:

SpikeInterface does not yet include a noise template classifier, but this is under active development. Until that is available, we recommend exporting the data into a format that can read by phy and manually labeling units with abnormal waveforms.

More information on manual curation be found in the documentation for Phy.

In [ ]:
import spikeinterface.full as si

from spikeinterface.postprocessing import (compute_spike_amplitudes,
                                           compute_principal_components)

from spikeinterface.exporters import export_to_phy

# the waveforms are sparse so it is faster to export to phy
we = si.extract_waveforms(recording=recording, sorting=sorting, folder='waveforms')

# compute some metrics needed for this module:
_ = compute_spike_amplitudes(waveform_extractor=we)
_ = compute_principal_components(waveform_extractor=we, 
                                 n_components=3, 
                                 mode='by_channel_global')

# save the data in a specified location
export_to_phy(waveform_extractor=we, 
              output_folder='path/to/phy_folder')

### 7) _mean_waveforms_:
Extracts mean waveforms from the raw data, given spike times and unit IDs. Also calculates metrics for each waveform.

Computes waveforms separately for individual epochs, as well as for the entire experiment. If no epochs are specified, waveforms are selected randomly from the entire recording. Waveform standard deviation is currently computed, but not saved.

Metrics are computed for every waveform, and include features of the 1D peak-channel waveform and the 2D waveform centered on the soma location.

1D waveform features: Waveform duration, peak-trough ratio, repolarization slope, and recovery slope.

2D waveform features: Waveform spread, velocity above the soma, and velocity below the soma.

#### SpikeInterface implementation:

SpikeInterface uses a __WaveformExtractor__ object to pull spike waveforms out of the raw data and compute metrics on their shape. More information can be found in the documentation for the Postprocessing module.

Extracting the mean waveforms from a sorting and computing a variety of waveform metrics only requires two lines of code:

In [ ]:
import spikeinterface.full as si

from spikeinterface.postprocessing import compute_template_metrics

waveform_extractor = si.extract_waveforms(recording=recording, 
                                          sorting=sorting, 
                                          folder='waveforms')

template_metrics = compute_template_metrics(waveform_extractor)
display(template_metrics)

### 8) _quality_metrics_:
Calculates quality metrics for each unit to assess isolation and sorting quality. Similar to the mean_waveforms module, this module can calculate metrics separately for individual epochs. If no epochs are specified, metrics are computed for the entire recording.

Included Metrics: Firing rate; Presence ratio; ISI violations; ISI violations (corrected); Amplitude cutoff; Isolation distance; L-ratio; d'; Nearest-neighbors; Silhouette score; Maximum drift; Cumulative drift.

#### SpikeInterface implementation:

All of the quality metrics in ecephys_spike_sorting (and more) have been ported to SpikeInterface. More information can be found in the documentation for the Quality Metrics module. Example code for computing quality metrics is shown here:

In [ ]:
import spikeinterface.full as si

from spikeinterface.qualitymetrics import compute_quality_metrics
from spikeinterface.postprocessing import compute_principal_components

# extract waveforms from a recording and sorting object
we = si.extract_waveforms(recording=recording, 
                          sorting=sorting, 
                          folder='waveforms')

# or load waveforms that have already been extracted:
we = si.load_waveforms(folder='waveforms')

# calculate metrics that don't require principal components:
metrics = compute_quality_metrics(waveform_extractor=we)

# or compute principal components before passing on the waveform extractor:
pca = compute_principal_components(waveform_extractor=we, 
                                   n_components=5, 
                                   mode='by_channel_local')
metrics = compute_quality_metrics(waveform_extractor=we)

### 9) _automerging_ (not used):

Searches for and automatically merges templates that likely belong to the same unit.

This module has not been used since switching to Kilosort 2, which has far fewer split units than Kilosort 1. We're keeping the code around in case others find it useful. For example, it could be helpful for matching units across a series of chronic recordings. However, much more complete implementations of this functionality exist elsewhere (UnitMatch, for example).

#### SpikeInterface implementation:

The SpikeInterface postprocessing module can compute information that is helpful for making merge decisions, such as waveform similarity and cross-correlograms:

In [ ]:
import spikeinterface.full as si

from spikeinterface.postprocessing import (compute_template_similarity,
                                           compute_correlograms)

# run a sorter and extract waveforms
# note that this omits some important pre-processing steps and parameters for brevity
recording = si.read_openephys('/path/to/data')
sorting = si.run_sorter('kilosort2_5', recording)
waveform_extractor = si.extract_waveforms(recording=recording, 
                                          sorting=sorting, 
                                          folder='waveforms')

# run post-processing steps
_ = compute_template_similarity(waveform_extractor)
_ = compute_correlograms(waveform_extractor)


In [ ]:
from spikeinterface.curation import get_potential_auto_merge

# Returns a list of unit pairs that should be merged
potential_merges = get_potential_auto_merge(waveform_extractor)

In addition, the curation module includes a function for identifying units that likely need to be merged. More information can be found in the documentation for the Curation module and the __get_potential_auto_merge__ method.